# Парсим мемы в питоне: как обойти серверную блокировку. 


# 1. Вламываемся в мемохранилище

## 1.1. Что мы хотим получить

Итак, мы хотим распарсить [knowyourmeme.com](http://knowyourmeme.com) и получить кучу разных переменных: 

- **Name** – название мема,
- **Origin_year** – год его создания,
- **Views** – число просмотров,
- **About** – текстовое описание мема,
- ** и многие другие** 

Более того, мы хотим сделать это без вот этого всего: 



После скачивания и чистки данных от мусора, можно будет заняться строительством моделей. Например, попытаться предсказать популярность мема по его параметрам. Но это все позже, а сейчас познакомимся с парой определений:

* **Парсер** — это скрипт, который грабит информацию с сайта
* **Краулер** — это часть парсера, которая бродит по ссылкам
* **Краулинг** — это переход по страницам и ссылкам
* **Скрапинг** — это сбор данных со страниц
* **Парсинг** — это сразу и краулинг и скрапинг! 


## 1.2.  Что такое HTML 

**HTML (HyperText Markup Language)**  — это такой же язык разметки как Markdown или LaTeX. Он является стандартным для написания различных сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта. 

Можно увидеть, что HTML-страница это ни что иное как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы 
- `<tr>` – разделитель для строк в таблице 
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега могут быть свои потомки (дети) — те теги, которые вложены в него, и свои родители. 

Например, HTML-древо страницы может выглядеть вот так:

    <html>
    <head> Заголовок </head>
    <body>
        <div> 
            Первый кусок текста со своими свойствами
        </div>
        <div>
            Второй кусок текста
                <b>
                    Третий, жирный кусок
                </b>
        </div>
        Четвёртый кусок текста        
    </body>
    </html>            
    
    


Можно работать с этим html как с текстом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Мы всего лишь будем находить нужные нам узлы среди всего этого разнообразия и забирать из них информацию!

Вручную обходить эти деревья не очень приятно, поэтому есть специальные языки для обхода деревьев.



## 1.3. Наш первый запрос

Доступ к веб-станицам позволяет получать модуль `requests`. Подгрузим его. За компанию подгрузим ещё парочку дельных пакетов. 

In [3]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек 
import time          # Библиотека для времени

Для наших благородных исследовательских целей нужно собрать данные по каждому мему с соответствующей ему страницы. Но для начала нужно получить адреса этих страниц. Поэтому открываем основную страницу со всеми выложенными мемами. Выглядит она следующим образом:



Со страницы всех мемов http://knowyourmeme.com/memes/all мы  будем тащить ссылки на каждый из перечисленных мемов. Сохраним в переменную `page_link` адрес основной страницы и откроем её при помощи библиотеки `requests`.

In [1]:
page_link = 'http://knowyourmeme.com/memes/all/page/1'

In [4]:
response = requests.get(page_link)
response

<Response [403]>

А вот и первая проблема! Обращаемся к [главному источнику знаний](https://en.wikipedia.org/wiki/HTTP_403) и выясняем, что 403-я ошибка выдается сервером, если он доступен и способен обрабатывать запросы, но по некоторым личным причинам отказывается это делать. 


Попробуем выяснить, почему. Для этого проверим, как выглядел финальный запрос, отправленный нами на сервер, а конкретнее - как выглядел наш User-Agent в глазах сервера.

In [5]:
for key, value in response.request.headers.items():
    print(key+": "+value)

User-Agent: python-requests/2.23.0
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive


Похоже, мы недвусмысленно дали понять серверу, что мы сидим на питоне и используем библиотеку requests под версией 2.14.2. Скорее всего, это вызвало у сервера некоторые подозрения относительно наших благих намерений и он решил нас безжалостно отвергнуть. 


Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными и пустить пыль в глаза сервера при помощи генерации фейкового юзер-агента. Библиотек, которые справляются с такой задачей, существует очень и очень много, лично мне больше всего нравится [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [7]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=b0c3deef5e4fd8c3c7e1fd1f5d572b385b04a01f2393a06139cd2f50cb99dae6
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [8]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [9]:
UserAgent().chrome

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2117.157 Safari/537.36'

In [10]:
UserAgent().safari

'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_7; da-dk) AppleWebKit/533.21.1 (KHTML, like Gecko) Version/5.0.5 Safari/533.21.1'

In [11]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

Замечательно, наша небольшая маскировка сработала и обманутый сервер покорно выдал благословенный 200 ответ — соединение установлено и данные получены, всё чудесно! Посмотрим, что же все-таки мы получили.

In [12]:
html = response.content

In [13]:
html[:1000]

b'<!DOCTYPE html>\n<html xmlns:fb=\'https://www.facebook.com/2008/fbml\' xmlns=\'https://www.w3.org/1999/xhtml\'>\n<head>\n<meta content=\'text/html; charset=utf-8\' http-equiv=\'Content-Type\'>\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHlcJWg==","queueTime":0,"applicationTime":58,"agent":""}</script>\n<script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam-cell.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"c1a6d52f38",applicationID:"31165848"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1

In [14]:
html[-1000:]

b'dToGoogleAnalytics); webVitals.getLCP(sendToGoogleAnalytics); }; document.getElementsByTagName("head")[0].appendChild(script); }()) </script>\n<div class=\'social-data\' data-media=\'https://s.kym-cdn.com/assets/kym-logo-large.png\' data-url=\'https://knowyourmeme.com/memes/all/page/1\'></div>\n<div class=\'complex-ad-wrapper adhesion hide\'> <div class=\'adhesion-dismissal\'> <a href="#" id="adhesion_dismiss">Close [X]</a> </div> <div class=\'complex-ad\' data-complex=\'false\' data-dfp-name=\'kym_adhesion_desktop\' data-dfp-size=\'[728, 90]\' data-options=\'{"containerId":"kym_adhesion","keywords":"userLoggedIn"}\' id=\'kym_adhesion\'></div> </div>\n<div class=\'complex-ad-wrapper\'> <div class=\'complex-ad\' data-complex=\'false\' data-dfp-name=\'kym_richmedia\' data-dfp-size=\'[1, 1]\' data-options=\'{"containerId":"kym_richmedia"}\' id=\'kym_richmedia\'></div> </div>\n<script async="" src="https://confiant-integrations.global.ssl.fastly.net/1LTHH6xASXMDHFRDP1KqXZl59ww/gpt_and_pr

In [15]:
len(html)

60316

In [16]:
type(html)

bytes

Выглядит неудобоваримо, как насчет сварить из этого дела что-то покрасивее? Например, красивый суп.

## 1.4. Красивый суп



Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** (тут есть гиперссылка на лучшего друга человека — документацию) был назван в честь стишка про красивый суп из Алисы в стране чудес.

Красивый суп — это совершенно волшебная библиотека, которая из сырого и необработанного HTML кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что нам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Нужно будет установить пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

С необработанным XML кодом страницы пакет также работает (XML — это исковерканый и превращённый в диалект, с помощью своих команд, HTML). Для того, чтобы пакет корректно работал с XML разметкой, придётся в довесок ко всему нашему арсеналу установить пакет `xml`. 

In [17]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы недавно получили.

In [18]:
soup = BeautifulSoup(html, 'html.parser') # В опции также можно указать lxml, 
                                         # если предварительно установить одноименный пакет

In [19]:
type(soup)

bs4.BeautifulSoup

In [20]:
print(soup.prettify()[:2000])

<!DOCTYPE html>
<html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={});NREUM.info={"beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHlcJWg==","queueTime":0,"applicationTime":58,"agent":""}
  </script>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam-cell.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"c1a6d52f38",applicationID:"31165848"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r

Получим что-то вот такое:
    
```
<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHkUNWUU=","queueTime":0,"applicationTime":24,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=
```

Стало намного лучше, не правда ли? Что же лежит в переменной `soup`? Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее, это не так. Теперь мы можем свободно бродить по HTML-дереву страницы, искать детей, родителей и вытаскивать их! 

Например, можно бродить по вершинам, указывая путь из тегов.

In [21]:
soup.html.head.title

<title>All Entries | Know Your Meme</title>

In [22]:
type(soup.html.head.title)

bs4.element.Tag

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [23]:
soup.html.head.title.text

'All Entries | Know Your Meme'

Более того, зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега `a` и имеет класс `photo`.

In [24]:
obj = soup.find('a', attrs = {'class':'photo'})
obj

<a class="photo left" href="/memes/i-said-we-dont-have-the-capacity" target="_self"><img alt="'We Don't Have The Capacity' Is A Hidden Gem From Last Year" data-src="https://i.kym-cdn.com/featured_items/icons/wide/000/014/548/S-zVECeBdPW0PJy1.jpeg" height="112" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="'We Don't Have The Capacity' Is A Hidden Gem From Last Year" width="198"/> <div class="info abs"> <div class="c"> 'We Don't Have The Capacity' Is A Hidden Gem From Last Year </div> </div> </a>

In [25]:
obj.get("class")

['photo', 'left']

Однако, вопреки нашим ожиданиям, вытащенный объект имеет класс `"photo left"`. Оказывается, `BeautifulSoup4` расценивает аттрибуты `class` как набор отдельных значений, поэтому `"photo left"` для библиотеки равносильно `["photo", "left"]`, а указанное нами значение этого класса `"photo"` входит в этот список. Чтобы избежать такой неприятной ситуации и проходов по ненужным нам ссылкам, придется воспользоваться собственной функцией и задать строгое соответствие:

In [26]:
'photo' in ['photo', 'left']

True

In [27]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
obj

<a class="photo" href="/memes/prayer-of-the-alt-knight"><img alt="Prayer Of The Alt-Knight" data-src="https://i.kym-cdn.com/entries/icons/medium/000/039/366/prayerofthealtknightmeme.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Prayer Of The Alt-Knight"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>

In [28]:
obj.get("class")

['photo']

In [29]:
type(obj)

bs4.element.Tag

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём! Вытащим ссылку на этот мем. Сделать это можно по атрибуту `href`, в котором лежит наша ссылка. 

In [30]:
obj.attrs['href']

'/memes/prayer-of-the-alt-knight'

Обратите внимание, что после всех этих безумных преобразований у данных поменялся тип. Теперь они `str`. Это означет, что с ними можно работать как с текстом и пускать в ход для отсеивания лишней информации регулярные выражения. 

In [31]:
print("Тип данных до вытаскивания ссылки:", type(obj))
print("Тип данных после вытаскивания ссылки:", type(obj.attrs['href']))

Тип данных до вытаскивания ссылки: <class 'bs4.element.Tag'>
Тип данных после вытаскивания ссылки: <class 'str'>


Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`, и на выход будет выдан список. Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с мемами.

In [32]:
meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
meme_links[:3]

[<a class="photo" href="/memes/prayer-of-the-alt-knight"><img alt="Prayer Of The Alt-Knight" data-src="https://i.kym-cdn.com/entries/icons/medium/000/039/366/prayerofthealtknightmeme.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Prayer Of The Alt-Knight"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>,
 <a class="photo" href="/memes/just-hay-fever"><img alt="Just Hay Fever tiktok trend and meme depicting a woman with a filter over her eyes making them seem extremely red, puffy and watery." data-src="https://i.kym-cdn.com/entries/icons/medium/000/039/365/hayfever.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Just Hay Fever"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>,
 <a class="photo" href="/memes/events/west-elm-caleb"><img alt="West Elm Caleb posts from TikTok videos from caleb, meemshou, kate g

In [33]:
len(meme_links)

16

Осталось очистить полученный список от мусора:

In [34]:
meme_links = [link.attrs['href'] for link in meme_links]

In [35]:
meme_links[:10]

['/memes/prayer-of-the-alt-knight',
 '/memes/just-hay-fever',
 '/memes/events/west-elm-caleb',
 '/memes/black-man-with-puffed-up-cheeks-holding-in-cough',
 '/memes/cristiano-ronaldo-winning-award',
 '/memes/cultures/squeecore',
 '/memes/it-is-here-that-we-harvest-half-of-the-french-salt',
 '/memes/you-must-be-the-belmont-trevor-punching-dracula',
 '/memes/gah-dayum',
 '/memes/stoned-typhlosion']

In [36]:
len(meme_links)

16

Готово, получили ровно 16 ссылок по числу мемов на одной странице поиска. 


Остался последний момент. Когда мы скачаем все мемы с текущей страницы, нам нужно будет каким-то образом забраться на соседнюю. На сайте это можно делать просто пролистывая страницу с мемами вниз, javascript-функции подтянут новые мемы на текущее окно, но сейчас трогать эти функции не хочется.

Обычно, все параметры, которые мы устанавливаем на сайте для поиска, отображаются на структуре хрефа. Мемы не являются исключением. Если мы хотим получить первую порцию мемов, мы должны будем обратиться к сайту по ссылке 

                `http://knowyourmeme.com/memes/all/page/1`


Если мы захотим получить вторую поцию с шестнадцатью мемами, нам придётся немного видоизменить ссылку, а именно заменить номер страницы на 2.


                `http://knowyourmeme.com/memes/all/page/2`
 
Таким незатейливым образом мы сможем пройтись по всем страницам и ограбить мемохранилище. 

Наконец, обернем в красивую функцию все-все манипуляции, проделанные выше:

In [37]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на мемы, полученный с текущей страницы
        
        page_number: int/string
            номер страницы для парсинга
            
    """
    # составляем ссылку на страницу поиска
    page_link = 'http://knowyourmeme.com/memes/all/page/{}'.format(page_number)
    
    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return [] 
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
    meme_links = ['http://knowyourmeme.com' + link.attrs['href'] for link in meme_links]
    
    return meme_links

Протестируем функцию и убедимся, что всё хорошо

In [38]:
meme_links = getPageLinks(1)
meme_links[:2]

['http://knowyourmeme.com/memes/prayer-of-the-alt-knight',
 'http://knowyourmeme.com/memes/just-hay-fever']

In [39]:
meme_links = getPageLinks(2)
meme_links[:2]

['http://knowyourmeme.com/memes/us-or-ps5',
 'http://knowyourmeme.com/memes/events/when-we-were-young-festival']

Отлично, функция работает и теперь мы теоретически можем достать ссылки на все мемы, для чего нам придется пройтись по 2000+ страниц. Прежде чем расстраивать сервер таким количеством запросов, посмотрим, как доставать всю необходимую информацию о конкретном меме. 

## 1.5 Финальная подготовка к грабежу

По аналогии со ссылками можно вытащить что угодно. Для этого надо сделать несколько шагов: 

1. Открываем страничку с мемом
2. Находим любым способом тег для нужной нам информации
3. Пихаем всё это в красивый суп
4. ......
5. Profit 

Для закрепления информации в голове любознательного читателя, вытащим число просмотров мема.

А в качестве примера возьмем самый популярный на этом сайте мем - Doge, набравший более 12 миллионов просмотров по состоянию на 1 января 2018 года. 

Сама страница, с которой мы будем доставать дорогую нашему исследовательскому сердцу информацию выглядит следуюшим образом:



Как и прежде, для начала сохраним ссылку на страницу в переменную и вытащим по ней контент.

In [40]:
meme_page = 'http://knowyourmeme.com/memes/doge'

response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

Посмотрим, как можно вытащить статистику просмотров, комментариев, а также числа загруженных видео и фото, связанных с нашим мемов. Всё это добро хранится справа вверху под тэгами `"dd"` и с классами  `"views"`, `"videos"`, `"photos"` и `"comments"`

In [41]:
views = soup.find('div', attrs={'class':'views'})
views

<div class="views" title="13,842,276 Views"></div>

In [42]:
type(views)

bs4.element.Tag

In [43]:
views = views.get("title")
views

'13,842,276 Views'

In [44]:
views.split()

['13,842,276', 'Views']

In [45]:
views = views.split()[0]
views

'13,842,276'

In [46]:
views = int(views.replace(',', ''))
views

13842276

Снова запихнём всё это в небольшую функцию.

In [47]:
def getStats(soup, stats):
    """
        Возвращает очищенное число просмотров/коментариев/...
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
            
        stats: string
            views/videos/photos/comments
            
    """
    try:
        obj = soup.find('div', attrs={'class':stats})
        obj = obj.get("title")
        obj = obj.split()[0]
        obj = int(obj.replace(',', ''))
    except:
        obj=None
    
    return obj

Всё готово! 

In [48]:
views = getStats(soup, stats='views')
videos = getStats(soup, stats='videos')
photos = getStats(soup, stats='photos')
comments = getStats(soup, stats='comments')

print("Просмотры: {}\nВидео: {}\nФото: {}\nКомментарии: {}".format(views, videos, photos, comments))

Просмотры: 13842276
Видео: 103
Фото: 1776
Комментарии: 925


Еще из интересного и исследовательского —  достанем дату и время добавления мема. Если посмотреть на страницу в браузере, можно подумать, что максимум информации, который мы можем вытащить - это число лет, прошедших с момента публикации —  `Added 4 years ago by NovaXP`. Однако мы так просто сдаваться не будем, полезем в кишки html и откопаем там кусок, ответственный за эту надпись:



Ага! Вот и подробности по дате добавления, с точностью до минуты. Элементарно

In [49]:
date = soup.findAll('abbr', attrs={'class':'timeago'})[1].attrs['title']
date

'2022-01-18T16:56:25-05:00'

На самом деле, парсеры — дело непредсказуемое. Часто страницы, которые мы парсим, имеют очень неоднородну структуру. Например, если мы парсим мемы, на части страниц может быть указано описание, а на части нет. Как только код впервые натыкается на отсутствие описания, он выдаёт ошибку и останавливается. Чтобы нормально собрать все данные, приходится прописывать исключения. Вроде бы, хранилище мемов хорошо оборудовано и никаких внештатных ситуаций происходить не должно. Тем не менее, очень не хочется проснуться утром и увидеть, что код сделал 20 итераций, нарвался на ошибку и отрубился.  Чтобы такого не произошло, можно, например, использовать конструкцию `try - except` и просто обрабатывать неугодные нам ошибки. Про исключения можно почитать [на просторах интернета](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html). В нашем же случае до ошибки можно и не доводить, а предварительно проверять, есть ли необходимый элемент на странице или нет при помощи обычного `if - else`, и уже после этого пытаться его распарсить.

Например, мы хотим вытащить статус мема, для этого найдем окружающие его тэги:

In [50]:
properties = soup.findAll('div', attrs={'class':'detail'})
properties[0]

<div class="detail">
<span>Status:</span>
<div class="tooltip-popup">
<span>
confirmed
</span>
</div>
</div>

In [51]:
prop = properties[0].findAll("span")
prop = [p.text.strip() for p in prop]
prop = {prop[0][:-1]:prop[1]}
prop

{'Status': 'confirmed'}

In [52]:
def _clean_property(prop):
    prop = prop.findAll("span")
    prop = [p.text.strip() for p in prop]
    prop = {prop[0][:-1]:prop[1]}
    return prop

In [53]:
for prop in properties:
    print(_clean_property(prop))

{'Status': 'confirmed'}
{'Origin': 'Tumblr'}
{'Year': '2010'}
{'Type': 'Character,\nAnimal,\nSlang'}


## Truthy & Falsy python 
https://www.freecodecamp.org/news/truthy-and-falsy-values-in-python/

In [54]:
if None:
    print(1)
else:
    print(2)

2


In [55]:
if [1, 2, 3]:
    print(1)

1


In [56]:
if []:
    print(1)

Такой код позволяет обезопасить себя от ошибок в коде. В данном случае, мы можем переписать всю конструкцию с `if - else` в виде одной удобной строки. Эта строка проверит полон ли респонса `meme_status` и ежели нет, то выдаст пустоту.

In [57]:
"Hello" if True else "Bye"

'Hello'

In [58]:
1 if ("a" in ['a', 'b']) or ('c' in [1, 4]) else 0

1

По аналогии можно вытащить всю остальную информацию со страницы.

In [59]:
def getProperties(soup):
    """
        Возвращает список (tuple) с названием, статусом, типом, 
        годом и местом происхождения и тэгами
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
    
    """
    # название - идёт с самым большим заголовком h1, легко найти
    meme_name = soup.find('section', attrs={'class':'info'}).find('h1').text.strip()
    
    clean_properties = {'Name':meme_name}
    properties = soup.findAll('div', attrs={'class':'detail'})
    for prop in properties:
        clean_properties.update(_clean_property(prop))
        
    return clean_properties

In [60]:
getProperties(soup)

{'Name': 'Doge Meme',
 'Origin': 'Tumblr',
 'Status': 'confirmed',
 'Type': 'Character,\nAnimal,\nSlang',
 'Year': '2010'}

Свойства мема собрали. Теперь собираем по аналогии его текстовое описание. 

In [61]:
def getText(soup):
    """
        Возвращает текстовые описания мема
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
            
    """
    
    # достаём все тексты под картинкой
    body = soup.find('section', attrs={'class':'bodycopy'})
    
    # раздел about (если он есть), должен идти первым, берем его без уточнения класса
    meme_about = body.find('p')
    meme_about = "" if not meme_about else meme_about.text
    
    # раздел origin можно найти после заголовка Origin или History, 
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin = body.find(text='Origin') or body.find(text='History')
    meme_origin = "" if not meme_origin else meme_origin.parent.find_next().text
    
    # весь остальной текст (если он есть) можно запихнуть в одно текстовое поле
    if body.text:
        other_text = body.text.strip().split('\n')[4:]
        other_text = " ".join(other_text).strip()
    else:
        other_text = ""
        
    return {'About':meme_about, 'Origin':meme_origin, 'Other_text':other_text}

In [62]:
text_fields = getText(soup)

print("О чем мем:\n{}\n\nПроисхождение:\n{}\n\nОстальной текст:\n{}...\n"\
      .format(text_fields['About'], text_fields['Origin'], text_fields['Other_text'][:200]))

О чем мем:
Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang term for "dog" that is primarily associated with pictures of Shiba Inus (nicknamed "Shibe") and internal monologue captions on Tumblr. These photos may be photoshopped to change the dog's face or captioned with interior monologues in Comic Sans font. Starting in 2017, Ironic Doge formats gained prevalence over the original wholesome version.

Происхождение:
The use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner's puppet show. In the episode titled "Biz Cas Fri 1"[2], Homestar calls Strong Bad his "d-o-g-e" while trying to distract him from his work.

Остальной текст:
OriginThe use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner's puppet show. In the episode titled "Biz Cas Fri 1"...



Наконец, создадим функцию, возвращающую всю информацию по текущему мему

In [63]:
def getMemeData(meme_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными
        
        meme_page: string
            ссылка на страницу с мемом
    
    """
    
    # запрашиваем данные по ссылке
    response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    views = getStats(soup=soup, stats='views')
    videos = getStats(soup=soup, stats='videos')
    photos = getStats(soup=soup, stats='photos')
    comments = getStats(soup=soup, stats='comments')

    # дата
    date = soup.findAll('abbr', attrs={'class':'timeago'})[1].attrs['title']

    # имя, статус, и т.д.
    properties = getProperties(soup=soup)

    # текстовые поля
    text_fileds = getText(soup=soup)

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"date_added":date, "views":views, 
                "videos":videos, "photos":photos, "comments":comments}
    data_row.update(properties)
    data_row.update(text_fileds)

    return data_row

In [64]:
data_row = getMemeData('http://knowyourmeme.com/memes/doge')

А теперь подготовим табличку, чтобы в неё записывать всё ~~награбленные~~ честно полученные данные, добавим в неё первую полученную строку и полюбуемся на результат

In [65]:
final_df = pd.DataFrame(columns=['Name', 'Status', 'Type', 'Year', 'Origin',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'About', 'Origin', 'Other_text'])

In [66]:
final_df = final_df.append(data_row, ignore_index=True)

In [67]:
final_df

,Name,Status,Type,Year,Origin,date_added,views,videos,photos,comments,About,Origin,Other_text
0,Doge Meme,confirmed,"Character,\nAnimal,\nSlang",2010,"The use of the misspelled word ""doge"" to refer...",2022-01-18T16:56:25-05:00,13842276,103,1776,925,Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang ter...,"The use of the misspelled word ""doge"" to refer...","OriginThe use of the misspelled word ""doge"" to..."


Первый мем оказался в наших рукак. Еще раз убедимся что всё работает — пройдемся по списку из ссылок на мемы, полученных ранее в перменной `meme_links`.

In [68]:
from tqdm import tqdm_notebook

In [69]:
meme_links

['http://knowyourmeme.com/memes/us-or-ps5',
 'http://knowyourmeme.com/memes/events/when-we-were-young-festival',
 'http://knowyourmeme.com/memes/kirby-fishing',
 'http://knowyourmeme.com/memes/mario-karted',
 'http://knowyourmeme.com/memes/goots',
 'http://knowyourmeme.com/memes/material-gworl',
 'http://knowyourmeme.com/memes/emotional-damage',
 'http://knowyourmeme.com/memes/trad-boywife-trad-husband',
 'http://knowyourmeme.com/memes/look-pim-i-know-its-our-job',
 'http://knowyourmeme.com/memes/20-fingers-gillette-short-dick-man',
 'http://knowyourmeme.com/memes/nyquil-chicken',
 'http://knowyourmeme.com/memes/events/ninja-vs-pokimane',
 'http://knowyourmeme.com/memes/events/microsofts-acquisition-of-activision-blizzard',
 'http://knowyourmeme.com/memes/subcultures/animusic',
 'http://knowyourmeme.com/memes/lost-in-the-fire-dance-challenge',
 'http://knowyourmeme.com/memes/nikocado-avocados-ass-pic']

In [70]:
for meme_link in tqdm_notebook(meme_links):
    try: 
        data_row = getMemeData(meme_link)
        final_df = final_df.append(data_row, ignore_index=True)
        time.sleep(0.3)
    except:
        continue

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/16 [00:00<?, ?it/s]

In [71]:
final_df.drop_duplicates(inplace=True)

In [72]:
final_df.shape

(17, 13)

In [73]:
final_df.head()

,Name,Status,Type,Year,Origin,date_added,views,videos,photos,comments,About,Origin,Other_text
0,Doge Meme,confirmed,"Character,\nAnimal,\nSlang",2010,"The use of the misspelled word ""doge"" to refer...",2022-01-18T16:56:25-05:00,13842276,103,1776,925,Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang ter...,"The use of the misspelled word ""doge"" to refer...","OriginThe use of the misspelled word ""doge"" to..."
1,Us or PS5? Meme,submission,"Participatory Media,\nViral Video",2021,"In November 2021, TikTok[1][2] dance and conte...",2022-01-20T20:37:57-05:00,2230,3,0,24,Us or PS5? or Them or PS5? refers to a viral v...,"In November 2021, TikTok[1][2] dance and conte...","OriginIn November 2021, TikTok[1][2] dance and..."
2,When We Were Young Festival Event,submission,"Convention,\nPerformance,\nPromotion",2022,,2022-01-21T04:59:41-05:00,3106,0,29,12,"The When We Were Young Festival, alternatively...",,"BackgroundOn January 18th, 2022, the official ..."
3,Kirby Fishing Meme,submission,"Exploitable,\nFan Art",2022,The trailer to Kirby and the Forgotten Land wa...,2022-01-20T19:33:15-05:00,253,3,11,0,Kirby Fishing is a scene from the trailer to K...,The trailer to Kirby and the Forgotten Land wa...,SpreadThe moment quickly inspired jokes about ...
4,Mario Karted Meme,submission,Viral Video,2014,"The original ""Get Mario Karted"" video was uplo...",2022-01-19T02:16:42-05:00,127,9,0,0,Mario Karted is a video game fail trend in whi...,"The original ""Get Mario Karted"" video was uplo...","OriginThe original ""Get Mario Karted"" video wa..."


Отлично! Всё работает, мемы качаются, данные наполняются и всё было бы хорошо, если бы не одно но — количество запросов, которое нам придётся сделать, чтобы всё получить.

# 2.  Прячемся от стражников

## 2.1 Когда работающий код больше не работает

Вот он! Тот самый момент абсолютного триумфа, когда код дописан и всё, что нам, мирным собирателям, остаётся — запустить наш код на одну ночку. Кажется, что через страсть мы преобрели силу. Запускаем наш код по всем  страницам с мемами. На всякий случай обернём наш цикл в `try-except`. Мало ли что там с этими мемами бывает. 

In [ ]:
# Немного красивых циклов. При желании пакет можно отключить и 
# удалить команду tqdm_notebook из всех циклов
from tqdm import tqdm_notebook


final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])

for page_number in tqdm_notebook(range(1500), desc='Pages'):
    # собрали хрефы с текущей страницы
    meme_links = getPageLinks(page_number)  
    for meme_link in tqdm_notebook(meme_links, desc='Memes', leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(3):
            try:
                # пытаемся собрать по мему немного даты
                data_row = getMemeData(meme_link)           
                # и закидываем её в таблицу
                final_df = final_df.append(data_row, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', meme_link)
                continue

C:\Users\mashk\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


Pages:   0%|          | 0/1783 [00:00<?, ?it/s]

C:\Users\mashk\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


Memes:   0%|          | 0/16 [00:00<?, ?it/s]

Memes:   0%|          | 0/16 [00:00<?, ?it/s]

Memes:   0%|          | 0/16 [00:00<?, ?it/s]

Memes:   0%|          | 0/16 [00:00<?, ?it/s]

Memes:   0%|          | 0/16 [00:00<?, ?it/s]

AHTUNG! parsing once again: http://knowyourmeme.com/memes/good-soldiers-follow-orders
AHTUNG! parsing once again: http://knowyourmeme.com/memes/events/drake-bell-child-endangerment
AHTUNG! parsing once again: http://knowyourmeme.com/memes/explain-in-familiar-terms
AHTUNG! parsing once again: http://knowyourmeme.com/memes/yes-rico-kaboom
AHTUNG! parsing once again: http://knowyourmeme.com/memes/yes-rico-kaboom
AHTUNG! parsing once again: http://knowyourmeme.com/memes/yes-rico-kaboom


Сон был прекрасным! Солнце только-только взошло из-за горизонта, мы уже бежим за компьютер смотреть мемы и видим, что огромное число мемов не скачалось.


Конечно же, вполне естественной реакцией будет нажать на первую же ссылку, перейти в мемохранилище и увидеть, что нас забанили.


Все наши реквесты остались без респонсов. 

## 2.2 Тор - сын Одина

Иногда серверу надоедает общаться с одним и тем же человеком, делающим кучу запросов и сервер банит его. К сожалению, не только у людей запас терпения ограничен.

Приходится маскироваться. Для такой маскировки можно использовать разные способы, более того, один из них мы уже использовали, когда притворились человеком в нашем `request-header`. Для текущей же задачи, когда нас вероломно заблокировали по IP, нужно искать способы помощнее, чтобы иметь возможность этот IP менять. Конечно, как вариант можно было бы использовать прокси-сервера, тогда мы бы имели в запасе некоторое количество разных IP адресов, которые можно подставлять по мере "забанивания". Однако в этом подходе есть пара проблем: первая - нужно где-то раздобыть эти прокси, вторая - а что если ограниченного числа адресов нам не хватит и нужно больше?

В таком случае лучше всего нам подойдёт [Tor](https://www.torproject.org/). Вопреки пропагандируемому мнению, Tor используется не только преступниками, педофилами и прочими нехорошими террористами. Это, мягко говоря, далеко не так, и мы, мирные собиратели данных, являемся тому подтверждением. Всем прелестям, связанным с работой Tor, можно было бы посвятить несколько больших статеек, что собственно говоря уже и сделано. Подробнее про это можно почитатать по следующим ссылкам: 

* [Как работает Tor](https://geektimes.ru/post/277578/)
* [Методы анонимности в сети](https://habrahabr.ru/post/204266/)
* [Прокси-сервер с помощью Tor](https://habrahabr.ru/company/etagi/blog/315002/)

Мы же ограничимся только функциональной частью, а именно без углубления в детали опишем шаги, которые нужно предпринять для того, чтобы использовать возможности Tor для обхода блокировки. Для начала полюбуемся на свой ip-адрес. Для этого сделаем get-запрос к сайту, который возвращяет наш IP

In [ ]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

In [ ]:
checkIP()

Current IP Address: 85.174.235.145


Заменить свой ip через Tor можно двумя путями. Простой - через браузер, а сложный - через небольшие махинации с настройками.

Скачаем браузерный [tor,](https://www.torproject.org/download/download) чтобы лёгкий путь был совсем прост. Для сложного пути в довесок к браузеру поставим tor через консоль. 
- Linux - нам поможет команда `apt-get install tor`, 
- Mac - сделаем это [в рамках brew](https://www.torproject.org/docs/tor-doc-osx.html.en), `brew install tor`. 
- Windows - ~~нам поможет установка другой операционной системы.~~ 

## 2.3 Путь первый 

Теперь запускаем свежескачанный браузер и оставляем его открытым.  Менять ip нам поможет библиотека `PySocks`. Конечно же, её нужно установить, скопировав в терминал `pip3 install PySocks`.  

Браузер тора по умолчанию использует порт номер 9150. В питоне при помощи библиотек socks и socket можно задать дефолтный порт для подключения. В результате текущая сессия будет использовать именно этот порт при отправке любого запроса, а значит – запросы будут посылаться из-под запущенного тора.

In [ ]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

Посмотрим на свой новый ip-aдрес. 

In [ ]:
checkIP()

Попробуем обратиться к мемохранилищу с нового ip-адреса.

In [ ]:
data_row = getMemeData('http://knowyourmeme.com/memes/doge')

for key, value in data_row.items():
    print(key.capitalize()+":", str(value)[:200], end='\n\n')

Бан снят. Стражники мемов ничего не заподозрили и пустили нас в сокровищницу. Чашу нашего респонса снова переполняет контент. Через силу мы обрели мощь. 

При желании, можно выяснить одну занимательную вещь: при базовых настройках, Тор-браузер меняет ip каждые 10 минут. Но что делать, если сервер банит нас быстрее?  Всё просто, в папке, куда был установлен Tor найдём файлик с настройками под названием torrc (на маке он лежит по адресу `~/Library/Application Support/TorBrowser-Data/torrc`, если не получится найти - добро пожаловать [сюда](https://tor.stackexchange.com/questions/11866/cant-find-torrc-file-on-mac)) и отредактируем его. Добавим строки: 

```
CircuitBuildTimeout 10
LearnCircuitBuildTimeout 0
MaxCircuitDirtiness 10
```

Минимально возможный период для обновления ip составляет 10 секунд. Установим туда эту цифру и попробуем поиграться. 

In [ ]:
for i in range(10):
    checkIP()
    time.sleep(5)

Current IP Address: 199.249.230.177
Current IP Address: 199.249.230.177
Current IP Address: 51.158.171.35
Current IP Address: 51.158.171.35
Current IP Address: 51.81.87.40
Current IP Address: 51.81.87.40
Current IP Address: 185.220.100.245
Current IP Address: 185.220.100.245
Current IP Address: 193.218.118.167
Current IP Address: 193.218.118.167


Действительно, смена ip происходит примерно раз в 10 секунд. Для наших целей по скачке мемов было достаточно и базовых настроек. Бан наступал примерно через 20 минут после начала работы кода. 

1. Открываем браузер;
2. Запускаем кусок кода с подгрузкой библиотек;
3. Запускаем цикл по мемам 
4. .....
5. Profit

In [ ]:
final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])


for page_number in tqdm_notebook(range(1455), desc='Pages'):
    # собрали хрефы с текущей страницы
    meme_links = getPageLinks(page_number)  
    for meme_link in tqdm_notebook(meme_links, desc='Memes', leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(5):
            try:
                # пытаемся собрать по мему немного даты
                data_row = getMemeData(meme_link)           
                # и закидываем её в таблицу
                final_df = final_df.append(data_row, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                continue
                
    final_df.to_csv('MEMES_{}.csv'.format(page_number))
    if page_number//10:
        print(final_df.shape)
final_df.to_csv('MEMES_full.csv')

Все мемы в наших руках. Можно приступать к варке фичей и моделированию. 

## 2.4 Путь второй - Stem

https://stem.torproject.org/

Stem is a Python controller library for Tor. With it you can use Tor's control protocol to script against the Tor process, or build things such as Nyx. Stem's latest version is 1.8 (released December 29th, 2019).

Позволяет подключаться к портам тора и более гибко настраивать взаимодействие с ним из питона

Теперь весь код для париснга можно перевести на рельсы этого небольшого скрипта, заменив все реквесты на торовские и менять IP адрес через каждые несколько реквестов к серверу, на который внезапно обрушится огромное число запросов из всех уголков света.

Напоследок, хотелось бы сказать пару слов о парсинге вообще и при помощи Тора в частности. Добывать себе данные - это стильно, модно и в принципе интересно, можно получить датасеты, которых еще никто никогда не обрабатывал, сделать что-то новое, посмотреть, наконец, на все мемы мира сразу. Однако не стоит забывать, что ограничения, введенные сервером, в том числе баны, появились не просто так, а в целях защиты сайта от недоброжелательных ковровых бомбардировок запросами и DDoS-атак. К чужому труду стоит относится с уважением, и даже если у сервера никакой защиты нет, - это еще не повод неограниченно забрасывать его реквестами, особенно если это может привести к его отключению - [уголовное наказание](http://sd-company.su/article/security/ddosataka-ugolovnaya-otvetstvennost) никто не отменял. Успешных и безопасных вам исследований!


## Почиташки 

* [Годная книга](https://github.com/FUlyankin/Parsers/blob/master/Ryan_Mitchell_Web_Scraping_with_Python-_Collecting_Data_from_the_Modern_Web_2015.pdf) про парсинг на английском языке. 
* [Неплохая инструкция](https://jarroba.com/anonymous-scraping-by-tor-network/) о самостоятельном парсинге через Tor без использования чужих готовых классов. 
* [Димин репозиторий](https://github.com/DmitrySerg/memology) с исследованием мемов.
* [Оригинальный кодекс](http://starwars.wikia.com/wiki/Code_of_the_Sith) адепта тёмной стороны силы.